In [0]:
!pip install mlxtend

In [0]:
import pandas as pd
import numpy as np
# Essemble
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from imblearn.ensemble import EasyEnsembleClassifier, RUSBoostClassifier
from sklearn.naive_bayes import GaussianNB #Naive Bayes
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import *

#ML steps structure
from sklearn.pipeline import FeatureUnion, Pipeline

#Preprocessing
from sklearn.preprocessing import FunctionTransformer, MinMaxScaler
from sklearn.feature_extraction.text import HashingVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD

from sklearn.model_selection import GridSearchCV

#Metrics
from sklearn.model_selection import cross_val_score

import io
from googleapiclient.http import MediaIoBaseDownload
import tensorflow as tf

import xgboost as xgb

import warnings

# Preprocessing

## Essemble

In [0]:
from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

In [0]:
folder = ''
response = drive_service.files().list(q=" '"+folder +"' in parents",
 spaces='drive',
 fields='nextPageToken, files(id, name)').execute()

In [0]:
datasets=[]

page_token = None
while True:
    response = drive_service.files().list(q="'"+ folder +"' in parents",
                                          spaces='drive',
                                          fields='nextPageToken, files(id, name)',
                                          pageToken=page_token).execute()
    for file in response.get('files', []):
        # Process change
        file_id = file.get('id')
        request = drive_service.files().get_media(fileId=file_id)
        downloaded = io.BytesIO()
        downloader = MediaIoBaseDownload(downloaded, request)
        done = False
        while done is False:
          # _ is a placeholder for a progress object that we ignore.
          # (Our file is small, so we skip reporting progress.)
          _, done = downloader.next_chunk()

        downloaded.seek(0)

        if('.csv' in file.get('name')):
          print(file.get('name'))
          datasets.append(io.BytesIO(downloaded.read()))

    page_token1 = response.get('nextPageToken', None)
    if page_token1 is None:
        break

X_.csv
y_.csv


In [0]:
X_train = pd.read_csv(datasets[0])
y_ = pd.read_csv(datasets[1])

In [0]:
X_values = X_train.loc[:, '0':].values

In [0]:
labels_data = pd.get_dummies(y_['0'])
col_names = list(range(0,11))
labels_data.columns = col_names
labels_data = labels_data.idxmax(axis=1)

### NB

In [0]:
base_estimator = GaussianNB()

In [0]:
scores_dt = {}
with tf.device('/gpu:0'):
  for n_estimator in [10, 15, 20]:
      print(n_estimator)
      eec = EasyEnsembleClassifier(base_estimator=base_estimator, n_jobs=-1,
                                   n_estimators=n_estimator, replacement=True)
      scores = cross_val_score(eec, X_values, labels_data.values, cv=10)
      print(scores)
      scores_dt['knn_'+str(n_estimator)] = scores

10
[0.11744777 0.11914173 0.12027103 0.13015246 0.12111801 0.11688312
 0.11998871 0.12111801 0.11857708 0.12450593]
15
[0.11942405 0.11744777 0.11942405 0.1230943  0.11998871 0.11801242
 0.11857708 0.12027103 0.11857708 0.12111801]
20
[0.11942405 0.11829475 0.12027103 0.12789385 0.11914173 0.11716544
 0.11829475 0.12083569 0.11801242 0.12281197]


### K-NN

In [0]:
base_estimator = KNeighborsClassifier(n_neighbors=7)

In [0]:
with tf.device('/gpu:0'):
  for n_estimator in [10, 15, 20]:
      print(n_estimator)
      eec = EasyEnsembleClassifier(base_estimator=base_estimator, n_jobs=-1,
                                   n_estimators=n_estimator, replacement=True)
      scores = cross_val_score(eec, X_values, labels_data.values, cv=10)
      print(scores)
      scores_dt['knn_'+str(n_estimator)] = scores

10


/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be 

[0.5417843  0.52117448 0.53444382 0.52371542 0.52879729 0.5214568
 0.53162055 0.52851496 0.51468097 0.5214568 ]
15


/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be 

[0.53896104 0.5214568  0.53867871 0.53218521 0.53303219 0.52540937
 0.53359684 0.52484472 0.51665726 0.52710333]
20


/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be 

[0.53444382 0.52484472 0.54206663 0.52597403 0.53839639 0.52230378
 0.5310559  0.53246753 0.52089215 0.5256917 ]


#### Decision tree

In [0]:
base_estimator = DecisionTreeClassifier(max_depth=25)

In [0]:
with tf.device('/gpu:0'):
  for n_estimator in [10, 15, 20]:
      print(n_estimator)
      eec = EasyEnsembleClassifier(n_estimators=n_estimator, n_jobs=-1,
                               base_estimator=base_estimator, replacement=True)
      scores = cross_val_score(eec, X_values, labels_data.values, cv=10, scoring='accuracy')
      print(scores)
      scores_dt['dt_'+str(n_estimator)] = scores

10


/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


[0.94635799 0.95369848 0.95426313 0.95906268 0.95426313 0.94889893
 0.94522868 0.94889893 0.95652174 0.95059289]
15


/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be 

[0.95652174 0.95482778 0.9539808  0.96329757 0.95567476 0.95200452
 0.95143986 0.95652174 0.95482778 0.95031056]
20
[0.95369848 0.96047431 0.95680407 0.96386222 0.95680407 0.95793337
 0.95031056 0.95736872 0.95708639 0.95454545]


### MLP

In [0]:
base_estimator = MLPClassifier(hidden_layer_sizes=(20,), max_iter=100, alpha=1e-4,
                    solver='sgd', tol=1e-4, random_state=42, momentum = 0.8, 
                    validation_fraction = 0.3, early_stopping = True, n_iter_no_change = 10,
                    learning_rate_init=.1, )

In [0]:
with tf.device('/gpu:0'):
  for n_estimator in [10, 15, 20]:
      print(n_estimator)
      eec = EasyEnsembleClassifier(n_estimators=n_estimator, n_jobs=-1,
                               base_estimator=base_estimator, replacement=True)
      scores = cross_val_score(eec, X_values, labels_data.values, cv=10)
      scores_dt['mlp_'+str(n_estimator)] = scores
      print(scores.mean())
      print(scores.std())

10
0.11699604743083003
0.007898278866318307
15
0.11369282891022021
0.005161324484493087
20
0.1153020892151327
0.007107899270938808


In [0]:
scores_dt

{'dt_10': array([0.94635799, 0.95369848, 0.95426313, 0.95906268, 0.95426313,
        0.94889893, 0.94522868, 0.94889893, 0.95652174, 0.95059289]),
 'dt_15': array([0.95652174, 0.95482778, 0.9539808 , 0.96329757, 0.95567476,
        0.95200452, 0.95143986, 0.95652174, 0.95482778, 0.95031056]),
 'dt_20': array([0.95369848, 0.96047431, 0.95680407, 0.96386222, 0.95680407,
        0.95793337, 0.95031056, 0.95736872, 0.95708639, 0.95454545]),
 'knn_10': array([0.5417843 , 0.52117448, 0.53444382, 0.52371542, 0.52879729,
        0.5214568 , 0.53162055, 0.52851496, 0.51468097, 0.5214568 ]),
 'knn_15': array([0.53896104, 0.5214568 , 0.53867871, 0.53218521, 0.53303219,
        0.52540937, 0.53359684, 0.52484472, 0.51665726, 0.52710333]),
 'knn_20': array([0.53444382, 0.52484472, 0.54206663, 0.52597403, 0.53839639,
        0.52230378, 0.5310559 , 0.53246753, 0.52089215, 0.5256917 ]),
 'mlp_10': array([0.10982496, 0.11547149, 0.11321287, 0.11067194, 0.11095426,
        0.12224732, 0.13749294, 0.116

In [0]:
df = pd.DataFrame(scores_dt)
df.to_csv('scores.csv')

In [0]:
for alg in ['dt', 'knn', 'nb', 'mlp']:
  mean = df.loc[:,df.columns.str.startswith(alg)].mean()
  std = df.loc[:,df.columns.str.startswith(alg)].std()

  print(alg)
  print(mean)
  print(mean.mean())
  print(std)
  print(std.mean())
  
for n in [10, 15, 20]:
  mean = df.loc[:,df.columns.str.endswith(n)].mean().mean()
  std = df.loc[:,df.columns.str.endswith(n)].mean().std()

  print(n)
  print(mean)
  print(std)

dt
dt_10    0.951779
dt_15    0.954941
dt_20    0.956889
dtype: float64
dt_10    0.004495
dt_15    0.003631
dt_20    0.003679
dtype: float64
knn
knn_10    0.526765
knn_15    0.529193
knn_20    0.529814
dtype: float64
knn_10    0.007876
knn_15    0.007326
knn_20    0.007041
dtype: float64
nb
Series([], dtype: float64)
Series([], dtype: float64)
mlp
mlp_10    0.110926
mlp_15    0.119481
mlp_20    0.115387
dtype: float64
mlp_10    0.005065
mlp_15    0.011936
mlp_20    0.003346
dtype: float64


KeyError: ignored